In [10]:
import csv
import os
import json

file = csv.reader(open('Data Journeys_ imports statements - ARCS_RULES.csv'), delimiter=',')
first = True
queries = []
prefixes = "PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#> \
PREFIX owl:  <http://www.w3.org/2002/07/owl#> \
PREFIX dj:   <http://purl.org/dj/> \
PREFIX : <http://purl.org/datajourneys/> \
"
# META QUERIES
# { ?x a :Input } <- { ?s :appearsIn [] . ?s :assignedFrom ?x }
queries.append(prefixes + " CONSTRUCT { ?x a :Input } WHERE { ?s dj:appearsIn [] . ?s dj:assignedFrom ?x }")
# { ?x a :Parameter } <- { ?s :appearsIn [] . FILTER NOT EXISTS { ?s :assignedFrom/:importedBy [] } }
queries.append(prefixes + " CONSTRUCT { ?x a :Parameter } WHERE { ?s dj:appearsIn [] . FILTER NOT EXISTS { ?s dj:assignedFrom/dj:importedBy [] }}")
# { ?x a :Support } <- { ?s :assignedFrom/:importedBy [] }
queries.append(prefixes + " CONSTRUCT { ?x a :Support } WHERE { ?s dj:assignedFrom/dj:importedBy [] }")
for line in file:
    if(first):
        first = False
        continue;
    arc = line[0]
#     print(arc[:-1])
    stype = line[3]
    ptype = line[4]
    otype = line[5]
    activity = line[6]
    if stype:
        queries.append(prefixes + " CONSTRUCT { ?s a %s } WHERE { ?s %s [] } " % (stype, arc))
    if ptype:
        queries.append(prefixes + " CONSTRUCT { %s rdfs:subPropertyOf %s } WHERE { [] %s [] } " % (arc, ptype, arc))
    if otype:
        queries.append(prefixes + " CONSTRUCT { ?o a %s } WHERE { [] %s ?o } " % (otype, arc))
    if activity:
        qa = prefixes + " CONSTRUCT { [] a %s ; rdf:subject ?s ; rdf:predicate ?p ; rdf:object ?o } WHERE { ?s " % activity
        qa += " ?s %s ?o . " % arc
        if stype:
            # add stype constraint
            qa += "?s a %s ." % stype
        if otype:
            # add otype constraint
            qa += "?o a %s ." % otype
        qa += "} "
#         print(qa)
        queries.append(qa)

with open('rules.json', mode='wt', encoding='utf-8') as myfile:
    json.dump(queries, myfile)